In [2]:
import pygame
import math
import numpy as np

field of vision in a basic env with one agent


In [12]:
import pygame

# Constants
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600
PREDATOR_RADIUS = 20
FOV_RADIUS = 100  # Field of vision radius
#  is the field of vision radius. It defines how far the 
# predator agent can "see" or sense its surroundings
# set to 100, meaning that the predator can sense objects 
# within a circular area with a radius of 100 pixels around itself. 
# used for visualization purposes. detect object within this range

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))


class Predator:
    def __init__(self, x, y):   #initialize
        self.x = x
        self.y = y

# just visualize the predator and its fov radius
    def draw(self):
        pygame.draw.circle(screen, RED, (int(self.x), int(self.y)), PREDATOR_RADIUS)
        pygame.draw.circle(screen, BLUE, (int(self.x), int(self.y)), FOV_RADIUS, 1)

def main():
    predator = Predator(200, 200)
    # specifying its initial position.

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        screen.fill(WHITE)
        predator.draw()
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


same code with movement and field of view in gym env


In [18]:
import pygame
import random

# Constants
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600
PREDATOR_RADIUS = 20
FOV_RADIUS = 100
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

class GameEnv:
    def __init__(self):
        # to make predator start from the middle of the screen
        # X AND Y ARE COORDINATES
        self.predator_x = SCREEN_WIDTH // 2
        self.predator_y = SCREEN_WIDTH // 2

    def draw_predator(self):
        pygame.draw.circle(screen, RED, (int(self.predator_x), int(self.predator_y)), PREDATOR_RADIUS)
        pygame.draw.circle(screen, BLUE, (int(self.predator_x), int(self.predator_y)), FOV_RADIUS, 1)

    def step(self, action):
        # Randomly change the predator's position within the screen
        if action == 0:  # Move up
            self.predator_y -= 2
        elif action == 1:  # Move down
            self.predator_y += 2
        elif action == 2:  # Move left
            self.predator_x -= 2
        elif action == 3:  # Move right
            self.predator_x += 2

        # Ensure the predator stays within the screen boundaries
        self.predator_x = max(PREDATOR_RADIUS, min(SCREEN_WIDTH - PREDATOR_RADIUS, self.predator_x))
        self.predator_y = max(PREDATOR_RADIUS, min(SCREEN_HEIGHT - PREDATOR_RADIUS, self.predator_y))

        observation = [self.predator_x, self.predator_y]
        reward = 0  
        done = False  
        info = {}  
        return observation, reward, done, info

    def reset(self):
        self.predator_x = 0
        self.predator_y = 0
        return [self.predator_x, self.predator_y]

def main():
    env = GameEnv()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        action = random.choice([0, 1, 2, 3])
        obs, reward, done, _ = env.step(action)

        screen.fill(WHITE)
        env.draw_predator()
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


added a wall as obstacle

In [20]:
import pygame
import random

# Constants
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600
PREDATOR_RADIUS = 20
FOV_RADIUS = 100
WALL_WIDTH = 20  # Width of the wall
WALL_HEIGHT = 40  # Width of the wall
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

class GameEnv:
    def __init__(self):
        self.predator_x = SCREEN_WIDTH // 2
        self.predator_y = SCREEN_HEIGHT // 2
        self.wall_x = SCREEN_WIDTH // 3
        self.wall_y = SCREEN_HEIGHT // 3

    def move_predator(self, action):
        # Randomly change the predator's position within the screen
        if action == 0:  # Move up
            self.predator_y -= 2
        elif action == 1:  # Move down
            self.predator_y += 2
        elif action == 2:  # Move left
            self.predator_x -= 2
        elif action == 3:  # Move right
            self.predator_x += 2

        # Ensure the predator stays within the screen boundaries
        self.predator_x = max(PREDATOR_RADIUS, min(SCREEN_WIDTH - PREDATOR_RADIUS, self.predator_x))
        self.predator_y = max(PREDATOR_RADIUS, min(SCREEN_HEIGHT - PREDATOR_RADIUS, self.predator_y))

        # Check for collision with the wall and adjust position if necessary
        # if (
        #     self.predator_x - PREDATOR_RADIUS < self.wall_x + WALL_WIDTH
        #     and self.predator_x + PREDATOR_RADIUS > self.wall_x
        #     and self.predator_y - PREDATOR_RADIUS < self.wall_y + WALL_HEIGHT
        #     and self.predator_y + PREDATOR_RADIUS > self.wall_y
        # ):
        #     # Reset the predator's position if it collides with the wall
        #     self.predator_x = SCREEN_WIDTH // 2
        #     self.predator_y = SCREEN_HEIGHT // 2

    def draw_environment(self):
        # Draw the wall obstacle
        pygame.draw.rect(screen, BLUE, (self.wall_x, self.wall_y, WALL_WIDTH, WALL_HEIGHT))

        # Draw the predator
        pygame.draw.circle(screen, RED, (int(self.predator_x), int(self.predator_y)), PREDATOR_RADIUS)
        pygame.draw.circle(screen, BLUE, (int(self.predator_x), int(self.predator_y)), FOV_RADIUS, 1)

    def step(self, action):
        self.move_predator(action)
        observation = [self.predator_x, self.predator_y]
        reward = 0  
        done = False  
        info = {}  
        return observation, reward, done, info

    def reset(self):
        self.predator_x = SCREEN_WIDTH // 2
        self.predator_y = SCREEN_HEIGHT // 2
        return [self.predator_x, self.predator_y]

def main():
    env = GameEnv()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
#  randomly selects an action from the list like move up or move down or left or right
        action = random.choice([0, 1, 2, 3])
        obs, reward, done, _ = env.step(action)

        screen.fill(WHITE)
        env.draw_environment()
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


field of view decreased to 50 degree from 360 degree using pygame.draw.arc function

In [29]:
import pygame
import random
import math

# Constants
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600
PREDATOR_RADIUS = 20
FOV_RADIUS = 100
FOV_ANGLE = 50  # Field of view angle in degrees
WALL_WIDTH = 20
WALL_HEIGHT = 40

WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

class GameEnv:
    def __init__(self):
        self.predator_x = SCREEN_WIDTH // 2
        self.predator_y = SCREEN_HEIGHT // 2
        self.wall_x = SCREEN_WIDTH // 3
        self.wall_y = SCREEN_HEIGHT // 3

    def move_predator(self, action):
        # Randomly change the predator's position within the screen
        if action == 0:  # Move up
            self.predator_y -= 2
        elif action == 1:  # Move down
            self.predator_y += 2
        elif action == 2:  # Move left
            self.predator_x -= 2
        elif action == 3:  # Move right
            self.predator_x += 2

        # Ensure the predator stays within the screen boundaries
        self.predator_x = max(PREDATOR_RADIUS, min(SCREEN_WIDTH - PREDATOR_RADIUS, self.predator_x))
        self.predator_y = max(PREDATOR_RADIUS, min(SCREEN_HEIGHT - PREDATOR_RADIUS, self.predator_y))

        # Check for collision with the wall and adjust position if necessary
        # if (
        #     self.predator_x - PREDATOR_RADIUS < self.wall_x + WALL_WIDTH
        #     and self.predator_x + PREDATOR_RADIUS > self.wall_x
        #     and self.predator_y - PREDATOR_RADIUS < self.wall_y + WALL_HEIGHT
        #     and self.predator_y + PREDATOR_RADIUS > self.wall_y
        # ):
        #     # Reset the predator's position if it collides with the wall
        #     self.predator_x = SCREEN_WIDTH // 2
        #     self.predator_y = SCREEN_HEIGHT // 2

    def draw_environment(self):
        # Draw the wall obstacle
        pygame.draw.rect(screen, BLUE, (self.wall_x, self.wall_y, WALL_WIDTH, WALL_HEIGHT))

        # Draw the predator
        pygame.draw.circle(screen, RED, (int(self.predator_x), int(self.predator_y)), PREDATOR_RADIUS)


###########################_??????????_##############################

        # Draw the FOV (Field of View)
        # FOV represents the area or cone within which the predator can see
        fov_points = [] #empty list that will store the points defining the FOV polygon.
        #  loop to iterate over a range of angles within the FOV. 
        # The range function is used to generate a sequence of angles.
        for angle in range(-FOV_ANGLE // 2, FOV_ANGLE // 2):
            #  loop calculates the coordinates (x and y) for each point on the FOV boundary 
            x = self.predator_x + FOV_RADIUS * math.cos(math.radians(angle))
            y = self.predator_y + FOV_RADIUS * math.sin(math.radians(angle))
            # math.cos and math.sin functions are used to calculate 
            # the x and y coordinates based on the angle. 
            fov_points.append((int(x), int(y)))
            # (x, y) point is then added to the fov_points list as a tuple of integers.
        pygame.draw.polygon(screen, BLUE, fov_points, 1)
        # pygame.draw.polygon is used to draw the FOV.
        # takes the screen as the drawing surface, 
        # BLUE as the color, 
        # fov_points as the list of points defining the polygon, 
        # and 1 as the width of the polygon lines. 
            # The result is that a polygon is drawn connecting the 
            # calculated points, representing the predator's FOV.
            
###########################_????????????_##############################


    def step(self, action):
        self.move_predator(action)
        observation = [self.predator_x, self.predator_y]
        reward = 0  
        done = False  
        info = {}  
        return observation, reward, done, info

    def reset(self):
        self.predator_x = SCREEN_WIDTH // 2
        self.predator_y = SCREEN_HEIGHT // 2
        return [self.predator_x, self.predator_y]

def main():
    env = GameEnv()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        action = random.choice([0, 1, 2, 3])
        obs, reward, done, _ = env.step(action)

        screen.fill(WHITE)
        env.draw_environment()
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()
